# 3.31 plan-1


# Linear Attention实现计划 (plan.md) - 修订版

## 目标

在现有的TorchTitan框架中实现线性注意力(Linear Attention)机制，并允许通过参数控制模型中常规注意力和线性注意力的混合比例。

## 背景

Linear Attention是一种减少注意力机制计算复杂度的方法，从O(n²)降低到O(n)，特别适合处理长序列。根据公式：

```
E(Q, K, V) = ρq(Q)(ρk(K)^T V)
```

我们将专注于Softmax实现方式:
- ρq(Y) = σrow(Y)：对每一行应用softmax
- ρk(Y) = σcol(Y)：对每一列应用softmax

关键点是先计算K和V的处理，再进行矩阵乘法，从而避免计算完整的注意力矩阵。

## 实现计划

### 1. 核心功能实现

1. **更新`linear_llama.py`中的线性注意力计算函数**
   - 实现`causal_linear_attention`函数，具体步骤：
     - 对K应用列softmax: ρk(K) = σcol(K)
     - 计算ρk(K)^T V，这是线性复杂度操作
     - 对Q应用行softmax: ρq(Q) = σrow(Q)
     - 计算ρq(Q)(ρk(K)^T V)，同样是线性复杂度
   - 确保因果性(causal)的正确实现，可能需要使用掩码或累积计算

2. **完善`LinearAttention`类**
   - 确保与现有的`Attention`类接口兼容
   - 实现前向传播逻辑，应用线性注意力计算

3. **完善`LinearTransformerBlock`类**
   - 确保它可以正确替代标准`TransformerBlock`

### 2. 混合架构实现

1. **创建`MixedTransformer`类**
   - 继承自`Transformer`
   - 接受一个新参数`linear_attn_ratio`控制线性注意力比例
   - 根据比例计算需要替换的层数

2. **实现层替换逻辑**
   - 计算需要替换的层数(`num_linear_layers`)
   - 确定替换位置，平均分布在整个网络中
   - 在这些位置使用`LinearTransformerBlock`，其它位置使用标准`TransformerBlock`

### 3. 配置扩展

1. **扩展`TransformerModelArgs`**
   - 添加`linear_attn_ratio`参数(默认为0，表示不使用线性注意力)
   - 去除之前提到的`linear_attn_type`参数，因为我们只使用softmax方法

2. **更新初始化逻辑**
   - 在`from_model_args`工厂方法中处理新参数
   - 根据参数确定使用哪种Transformer实现

### 4. 训练和评估支持

1. **更新并行化和管道逻辑**
   - 修改`parallelize_llama.py`以支持混合架构
   - 确保梯度检查点(checkpoint)功能正常工作

2. **添加评估指标**
   - 测量不同`linear_attn_ratio`值下的性能和准确性变化

## 技术挑战与注意事项

1. **因果性保证**
   - 在线性注意力中实现因果性是关键挑战
   - 可能需要使用累积计算或特殊掩码技术

2. **数值稳定性**
   - 在应用softmax时添加必要的epsilon值防止数值问题
   - 考虑在数值可能溢出的地方使用log-space计算

3. **内存效率**
   - 确保实现真正避免了O(n²)的内存需求
   - 注意中间结果的存储方式

4. **性能优化**
   - 确保实现充分利用了矩阵运算的并行性
   - 考虑在长序列上分块处理

## 测试计划

1. **单元测试**
   - 测试线性注意力计算的正确性
   - 验证因果掩码的正确应用
   - 比较线性注意力和标准注意力在小规模输入上的输出差异

2. **集成测试**
   - 测试不同`linear_attn_ratio`值的端到端训练
   - 验证混合模型的推理结果

3. **性能基准**
   - 测量不同线性注意力比例的训练和推理速度
   - 评估在长序列上的内存使用和计算效率提升

## 下一步

1. 实现`causal_linear_attention`计算核心，专注于行列softmax方法
2. 更新`LinearAttention`和`LinearTransformerBlock`类
3. 实现`MixedTransformer`支持混合架构
4. 添加配置参数并更新模型创建逻辑
5. 进行测试和性能评估


# 3.31 plan-2


# Linear Attention 训练与评估计划

## 1. 训练配置准备

1. **创建不同线性注意力比例的配置文件**
   - 基于 `llama3_1b.toml` 创建以下配置文件:
     - `llama3_1b_linear_0.25.toml` (25% 线性注意力)
     - `llama3_1b_linear_0.50.toml` (50% 线性注意力)
     - `llama3_1b_linear_0.75.toml` (75% 线性注意力)
     - `llama3_1b_linear_1.00.toml` (100% 线性注意力)

2. **配置文件修改**
   - 在每个配置文件中添加 `linear_attn_ratio` 参数:
   ```toml
   [model]
   name = "llama3"
   flavor = "1B"
   linear_attn_ratio = 0.XX  # 根据配置调整
   ```
   - 为每个配置设置不同的输出目录:
   ```toml
   [job]
   dump_folder = "./outputs/llama3_1b_linear_0.XX"
   description = "Llama 3 1B with XX% linear attention"
   ```

3. **参数优化考虑**
   - 根据线性注意力的内存效率，考虑调整以下参数:
     - 批量大小 (`batch_size`): 可能可以增加
     - 序列长度 (`seq_len`): 考虑使用更长序列
     - 学习率: 可能需要针对线性注意力调整

## 2. 训练脚本适配

1. **确保模型构建正确使用 `MixedTransformer`**
   - 检查模型构建流程，确保 `linear_attn_ratio` 参数被正确传递
   - 确认使用的是 `MixedTransformer` 而非标准 `Transformer`

2. **添加日志和监控**
   - 添加专门记录线性注意力层信息的日志
   - 在训练日志中明确标识线性注意力比例
   - 添加内存使用监控，特别关注注意力计算部分

3. **检查点兼容性**
   - 确保检查点保存/加载机制支持 `LinearTransformerBlock`
   - 添加模型架构信息到检查点元数据

## 3. 执行训练

1. **初始小规模测试**
   - 先用小数据集和较少步数测试所有配置，确保训练稳定性
   - 使用命令如:
   ```bash
   python train.py --config train_configs/llama3_1b_linear_0.25.toml
   ```

2. **完整训练执行**
   - 对每个线性注意力比例执行完整训练
   - 使用相同的随机种子以确保公平比较
   - 记录以下指标:
     - 训练吞吐量(tokens/second)
     - GPU 内存使用
     - 训练损失曲线
     - 每轮训练时间

3. **训练过程监控**
   - 使用 TensorBoard/WandB 跟踪不同配置的训练过程
   - 设置关键指标警报，及早发现训练问题

## 4. 性能评估

1. **基础指标评估**
   - 对所有模型变体测量以下指标:
     - 困惑度(Perplexity)
     - 训练和验证损失
     - 推理速度(不同输入长度)
     - 内存使用效率

2. **长序列能力测试**
   - 测试不同模型在超长输入(8K-32K tokens)上的性能
   - 评估注意力机制在长序列上的质量差异
   - 测量不同序列长度下的内存使用曲线

3. **下游任务评估**
   - 在标准基准测试集上评估不同模型:
     - MMLU (常识推理)
     - HumanEval (编码能力)
     - GSM8K (数学推理)
   - 特别关注需要长距离依赖的任务

4. **生成质量比较**
   - 进行人工评估，比较不同模型生成文本的质量
   - 使用 10-20 个提示词，评估生成的连贯性、准确性和创造性

## 5. 注意力机制分析

1. **注意力模式可视化**
   - 为标准注意力和线性注意力层创建注意力热图
   - 分析不同层级的注意力分布差异
   - 研究长距离依赖的捕获能力

2. **不同线性比例的影响分析**
   - 绘制线性注意力比例与模型性能的关系曲线
   - 分析不同层使用线性注意力的影响
   - 探索最佳的线性注意力分布策略

3. **计算复杂度分析**
   - 测量不同序列长度下的计算时间增长曲线
   - 验证线性注意力的理论复杂度优势
   - 分析瓶颈和潜在的优化机会

## 6. 优化与改进

1. **分布策略优化**
   - 测试不同的线性注意力层分布策略:
     - 均匀分布
     - 集中在特定位置(前部/中部/后部)
     - 基于层功能的自适应分配

2. **混合精度训练优化**
   - 分析线性注意力在不同精度下的数值稳定性
   - 测试 float16/bfloat16 下的性能
   - 优化梯度缩放策略

3. **超参数调优**
   - 针对线性注意力模型调整关键超参数:
     - 学习率
     - 优化器参数
     - 注意力温度或缩放因子


# 4.2 plan


# 线性注意力实现问题修复计划

## 一、问题诊断

1. **不正确的注意力机制实现**
   - 当前`causal_linear_attention`函数实现中的softmax应用不当，导致模型快速塌陷到平凡解
   - 因果掩码实现不完整，导致信息泄露
   - 块处理实现存在数值稳定性问题

2. **标准注意力与线性注意力对比**
   - 标准注意力使用`F.scaled_dot_product_attention`，而线性注意力使用自定义实现
   - 缺少缩放因子导致数值不稳定
   - 线性近似实现不正确，不能正确模拟标准注意力的行为

## 二、修复方案
https://github.com/cmsflash/efficient-attention/blob/master/efficient_attention.py 这是linear attention提供的官方实现代码，可以作为参考，但是我们使用的是casual，并且并不是图像，而是文字，参考时候要注意区别

1. **修改因果线性注意力实现**
   - 重新实现`causal_linear_attention`函数：
     - 添加正确的缩放因子 (类似标准注意力的`1/sqrt(head_dim)`)
     - 修改掩码策略确保严格因果关系
     - 修复softmax应用维度

2. **改进线性注意力近似方法**
   - 实现基于核方法的线性注意力
   - 添加归一化因子保持数值稳定性
   - 增加梯度缩放以确保稳定训练

3. **引入训练稳定性措施**
   - 添加注意力权重的正则化
   - 实现线性注意力的梯度蒸馏
   - 在训练初期使用混合注意力机制进行平滑过渡

4. **增强模型监控**
   - 添加注意力层的梯度和激活值监控
   - 实现注意力分布的可视化工具
   - 为线性注意力实现添加数值稳定性检查

## 三、测试与评估

1. **单元测试**
   - 验证线性注意力与标准注意力的数学等价性
   - 测试因果掩码的正确性
   - 检测长序列下的注意力行为

2. **集成测试**
   - 验证不同`linear_attn_ratio`值下模型的收敛性
   - 测试参数更新的稳定性
   - 对比不同实现在推理和训练中的差异

3. **性能评估**
   - 对比两种注意力机制的计算效率
   - 测量内存使用和吞吐量
   - 评估损失曲线的收敛速度和稳定性

## 四、实施时间表

1. **第一阶段：核心修复**
   - 修复`causal_linear_attention`函数的基本问题
   - 实现初步的单元测试评估

2. **第二阶段：增强与稳定**
   - 添加正则化和梯度控制机制
   - 实现监控工具与诊断功能

3. **第三阶段：优化与扩展**
   - 基于测试结果进行性能优化
   - 进一步改进注意力方法，探索混合注意力机制


# 4.3 plan

今日执行计划
## 1. 修复线性注意力实现
### 修改 causal_linear_attention 函数:
参照 lucidrains 的实现，重写核心计算逻辑
专注于矩阵乘法顺序优化和数值稳定性
使用适当的缩放和注意力激活函数
### 调整 LinearAttention 类:
确保与更新的注意力函数兼容
改进初始化和归一化方法
### 修复内存管理:
优化大序列处理中的内存使用
改进因果掩码实现
## 2. 单节点测试验证
### 使用小型配置快速测试:
创建精简版本的配置文件用于快速验证
监控损失曲线收敛性和梯度流动
### 运行基准测试:
比较修复前后的性能和内存使用
验证修复的实现与标准注意力在行为上的一致性
## 3. 多节点线性注意力比例实验
### 准备实验配置:
创建 linear_ratio=0.25 配置文件
创建 linear_ratio=0.5 配置文件
创建 linear_ratio=1.0 配置文件
### 执行长期训练:
使用相同种子进行训练以确保可比性
收集每个配置的训练和推理性能指标